In [1]:
import flopy
import numpy as np

# Define model domain and grid
model_name = "groundwater_transport_model"
model_dir = "path/to/model/directory"
nlay = 1  # Number of layers
nrow = 50  # Number of rows
ncol = 50  # Number of columns
delr = 40  # Grid spacing in meters (east-west)
delc = 40  # Grid spacing in meters (north-south)
top_elevation = 192.0  # m.a.s.l at the southwestern corner
bot_elevation = 184.0  # m.a.s.l at the northern boundary of the model

# Create a new MODFLOW model
model = flopy.modflow.Modflow(model_name, model_ws=model_dir)

# Create DIS package
dis = flopy.modflow.ModflowDis(
    model, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc,
    top=top_elevation, botm=bot_elevation, perlen=1
)

# Define properties for the underlying clay layer
clay_layer_top = 188.0
clay_layer_bottom = 177.0

# Create BAS package (basic package)
bas = flopy.modflow.ModflowBas(model, ibound=1, strt=top_elevation)

# Create LPF package (layer property flow package)
lpf = flopy.modflow.ModflowLpf(
    model, hk=1.0, vka=1.0, laytyp=1, layavg=0, chani=1.0, ss=1e-5
)

# Create OC package (output control package)
oc = flopy.modflow.ModflowOc(model)

# Create PCG package (preconditioned conjugate gradient package)
pcg = flopy.modflow.ModflowPcg(model)

# Create the basic transport package (BTN)
btn = flopy.mt3d.Mt3dBtn(
    model, laycon=0, mixelm=0, percel=1, ncomp=1, nprs=1,
    dt0=0.001, ttsmult=1.0, ttsmax=0
)

# Create the advection package (ADV)
adv = flopy.mt3d.Mt3dAdv(model, mixelm=0)

# Create the dispersion package (DSP)
dsp = flopy.mt3d.Mt3dDsp(model, al=0.0, trpt=1.0, dmcoef=0.0)

# Create the source/sink mixing package (SSM)
ssm_data = {0: [[0, 25, 25, 0.0]]}  # Injecting a mass at cell (0, 25, 25)
ssm = flopy.mt3d.Mt3dSsm(model, sources=ssm_data)

# Write the input files and run the model
model.write_input()
success, mfoutput = model.run_model(silent=True)

# Post-process the results if needed
# For example, visualize concentration distribution over time using MT3DMS output files

print("Model run complete.")


Exception: BTN error. Required input is None, but no modflow model. If no modflow model is passed to Mt3dms, then values must be specified in the BTN constructor for: nlay, nrow, ncol, nper, laycon, delr, delc, htop, dz, perlen, nstp, and tsmult.